# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""10077498185059855319""",5.7611e18,"""../../../../data/SampleHDS.jso…","""10656365717583539913""","""10261671901979445030""","""7750833398573795376""",false
"""942729983063710018""",1.7026e19,"""../../../../data/SampleHDS.jso…","""6607561431314487482""","""10261671901979445030""","""14123790688265052903""",true
"""3818815615138386383""",2.9634e18,"""../../../../data/SampleHDS.jso…","""8037215748665178806""","""11284186782676436401""","""7750833398573795376""",false
"""813335576182279504""",9.6018e18,"""../../../../data/SampleHDS.jso…","""6607561431314487482""","""8629175929929579770""","""7750833398573795376""",false
"""11137938681377716231""",1.1695e19,"""../../../../data/SampleHDS.jso…","""10656365717583539913""","""8629175929929579770""","""1894772632572944101""",true
"""6486769435483843006""",3.2391e18,"""../../../../data/SampleHDS.jso…","""8037215748665178806""","""5510782863652847641""",null,false
"""12907511846830711410""",1.2508e19,"""../../../../data/SampleHDS.jso…","""8037215748665178806""","""8629175929929579770""","""1894772632572944101""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'Customer_CLV': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""17021200390789634767""",1.7319e19,"""../../../../data/SampleHDS.jso…","""8973579136740895916""","""FirstMortgage30yr""","""8871676171556147795""","""Rejected"""
"""17544086720894178852""",1.2311e19,"""../../../../data/SampleHDS.jso…","""17338410244790542383""","""FirstMortgage30yr""","""12151606643379405297""","""Accepted"""
"""2608404415705092858""",1.7058e19,"""../../../../data/SampleHDS.jso…","""12204139172891107800""","""MoneyMarketSavingsAccount""","""8871676171556147795""","""Rejected"""
"""5817609737565267353""",4.1345e18,"""../../../../data/SampleHDS.jso…","""17338410244790542383""","""BasicChecking""","""8871676171556147795""","""Rejected"""
"""17215557615437904129""",1.3507e19,"""../../../../data/SampleHDS.jso…","""8973579136740895916""","""BasicChecking""","""6018071385505420650""","""Accepted"""
"""8151885844329759850""",1.7773e19,"""../../../../data/SampleHDS.jso…","""12204139172891107800""","""UPlusFinPersonal""",null,"""Rejected"""
"""4576154232532392396""",2.3662e18,"""../../../../data/SampleHDS.jso…","""12204139172891107800""","""BasicChecking""","""6018071385505420650""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""813076299930669557""",8.4757e17,"""../../../../data/SampleHDS.jso…","""13730240430822337523""","""FirstMortgage30yr""","""1547786191524735015""","""Rejected"""
"""5705327515575178129""",3.3800e18,"""../../../../data/SampleHDS.jso…","""5021490259315704636""","""FirstMortgage30yr""","""18370968546890038233""","""Accepted"""
"""8045349805967082244""",9.4535e18,"""../../../../data/SampleHDS.jso…","""5829952120196273636""","""MoneyMarketSavingsAccount""","""1547786191524735015""","""Rejected"""
"""8040904595279708011""",1.3146e19,"""../../../../data/SampleHDS.jso…","""5021490259315704636""","""BasicChecking""","""1547786191524735015""","""Rejected"""
"""12474447813230399459""",1.9948e18,"""../../../../data/SampleHDS.jso…","""13730240430822337523""","""BasicChecking""","""5088144089422412893""","""Accepted"""
"""11984387239614018734""",4.4509e18,"""../../../../data/SampleHDS.jso…","""5829952120196273636""","""UPlusFinPersonal""",null,"""Rejected"""
"""17736879331245226295""",1.3874e19,"""../../../../data/SampleHDS.jso…","""5829952120196273636""","""BasicChecking""","""5088144089422412893""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
Customer_CLV=PREDICTOR_1
filename=filename
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""11886092612887717183""",1.1897e18,"""8147083483650163615""","""FirstMortgage30yr""","""5190834302216795803""","""Rejected"""
"""1094177178592495301""",1.0869e19,"""14057640247994721063""","""FirstMortgage30yr""","""222441260287063194""","""Accepted"""
"""10330824417150314753""",3.5389e18,"""8088142751856650856""","""MoneyMarketSavingsAccount""","""5190834302216795803""","""Rejected"""
"""7899999843716291045""",1.1766e19,"""14057640247994721063""","""BasicChecking""","""5190834302216795803""","""Rejected"""
"""17524805312059947946""",1.0191e19,"""8147083483650163615""","""BasicChecking""","""13141097939005242326""","""Accepted"""
"""874051649157329442""",5.6537e18,"""8088142751856650856""","""UPlusFinPersonal""",null,"""Rejected"""
"""17051969167667844559""",1.0532e19,"""8088142751856650856""","""BasicChecking""","""13141097939005242326""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'